# Environment Setting

- MusicVAE [Colab Notebook](https://colab.research.google.com/github/magenta/magenta-demos/blob/master/colab-notebooks/MusicVAE.ipynb#scrollTo=0x8YTRDwv8Gk)을 참고하여 세팅

In [2]:
import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
tf.enable_eager_execution()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155632 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4 MB 18.2 MB/s 
     |████████████████████████████████| 352 kB 31.1 MB/s 
     |████████████████████████████████| 1.6 MB 35.4 MB/s 
     |████████████████████████████████| 90 kB 8.6 MB/s 
     |████████████████████████████████| 254 kB 54.6 MB/s 
     |███████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


# Preprocess MIDI to tfrecord

In [3]:
import os
import hashlib

from note_seq import abc_parser
from note_seq import midi_io
from note_seq import musicxml_reader
import tensorflow.compat.v1 as tf

## Step1. Convert MIDI to Proto Type of Sequence

- midi 형식의 파일을 proto 형식의 sequence로 변환

In [4]:
cd /content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts

/content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts


In [7]:
sequence = midi_io.midi_to_sequence_proto(tf.gfile.GFile('/content/drive/MyDrive/Colab Notebooks/samples/Trained_epoch_50058_groovae_4bar_sample_0.mid', 'rb').read())
sequence

ticks_per_quarter: 220
time_signatures {
  numerator: 4
  denominator: 4
}
tempos {
  qpm: 120.0
}
notes {
  pitch: 42
  velocity: 17
  end_time: 0.12272727272727273
  is_drum: true
}
notes {
  pitch: 42
  velocity: 37
  start_time: 0.12272727272727273
  end_time: 0.2477272727272727
  is_drum: true
}
notes {
  pitch: 42
  velocity: 107
  start_time: 0.23636363636363636
  end_time: 0.2477272727272727
  is_drum: true
}
notes {
  pitch: 42
  velocity: 35
  start_time: 0.36818181818181817
  end_time: 0.49318181818181817
  is_drum: true
}
notes {
  pitch: 38
  velocity: 127
  start_time: 0.4909090909090909
  end_time: 0.6159090909090909
  is_drum: true
}
notes {
  pitch: 42
  velocity: 34
  start_time: 0.6181818181818182
  end_time: 0.7431818181818182
  is_drum: true
}
notes {
  pitch: 42
  velocity: 106
  start_time: 0.7363636363636363
  end_time: 0.7431818181818182
  is_drum: true
}
notes {
  pitch: 42
  velocity: 104
  start_time: 0.8727272727272727
  end_time: 0.9977272727272727
  is_dr

In [8]:
play(sequence)

## Step2. Convert Sequence to TFRecord

- `tfrecord`: tensorflow에서 사용할 수 있는 바이너리 형식
- TFRecordDataset 모듈을 통해 데이터셋 구축 가능
- /content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts에 `test_tfrecord` 저장

In [9]:
with tf.io.TFRecordWriter('test_tfrecord') as writer:
    writer.write(sequence.SerializeToString())

## Step3. Convert all MIDI files in Input_Directory to Sequence_TFRecord

- `convert_dir_to_note_sequences.py`: 폴더 내 모든 midi 파일을 tfrecord 형식으로 변환
- 전처리과정 `Step1` & `Step2`를 반복하며 폴더 내 모든 midi파일을 tfrecord 형식으로 변환
- `samples` 폴더를 읽어 midi 파일을 `tfrecord` 형식으로 변환
- /content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts에 `SEQUENCES_TFRECORD` 저장

In [14]:
cd /content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts

/content/drive/MyDrive/Colab Notebooks/magenta/magenta/scripts


In [12]:
!python convert_dir_to_note_sequences.py \
  --input_dir="/content/drive/MyDrive/Colab Notebooks/samples" \
  --output_file=SEQUENCES_TFRECORD \
  --recursive

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Instructions for updating:
non-resource variables are not supported in the long term
2022-06-05 20:33:16.348403: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-cap

In [13]:
raw_dataset = tf.data.TFRecordDataset('SEQUENCES_TFRECORD')
for raw_record in raw_dataset.take(5):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n9/id/midi/samples/a2b23823080225edf5c6f902707c979dde2953e1\x12-Trained_epoch_40106_groovae_4bar_sample_2.mid\x1a\x07samples \xdc\x01*\x04\x10\x04\x18\x04:\t\x11\x00\x00\x00\x00\x00\x00^@B\x18\x08*\x108\x19k\xdf\xb0\xf6\rk\xdf?!\xb5oX\xfb\x86\xb5\xe3?H\x01B\x18\x08$\x10\x1d\x19\xc8S\x82<%\xc8\xef?!\xe4)A\x9e\x12\xe4\xf1?H\x01B\x18\x08&\x105\x19\xe4)A\x9e\x12\xe4\xf7?!\xe4)A\x9e\x12\xe4\xf9?H\x01B\x18\x08*\x10\x1c\x19\xed\x1b\xd6\xbea\xed\xf7?!\xed\x1b\xd6\xbea\xed\xf9?H\x01B\x18\x08$\x10\x1e\x19\xc8S\x82<%\xc8\xff?!\xe4)A\x9e\x12\xe4\x00@H\x01B\x18\x08*\x10C\x19\xe4)A\x9e\x12\xe4\x03@!\xe4)A\x9e\x12\xe4\x04@H\x01B\x18\x08$\x10+\x19\xdf\xb0\xf6\rk\xdf\x07@!\xdf\xb0\xf6\rk\xdf\x08@H\x01B\x18\x08&\x10\x1d\x19\xf6\rk\xdf\xb0\xf6\x0b@!\xf6\rk\xdf\xb0\xf6\x0c@H\x01B\x18\x08$\x10'\x19\xd6\xbea\xed\x1b\xd6\x0f@!k\xdf\xb0\xf6\rk\x10@H\x01B\x18\x08*\x10\x1e\x19t\xd1E\x17]t\x11@!t\xd1E\x17]\xf4\x11@H\x01B\x18\x08&\x10S\x19\xf6\rk\xdf\xb0\xf6\x11@!\xf6\